In [ ]:
%matplotlib inline
# import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ggplot import *
plt.style.use('ggplot')

# Code4Lib 2017 Workshop Part 2

## Merging Data

In many real world situations, the data we'd like to analyze comes in multiple files. Pandas provides multiple different ways of combining data, including "merge()", and "concat()". To get started, we first need to load some datasets into DataFrames. We'll use the same surveys data as in the last part of the workshop, as well as another dataset called "surveys2.csv", which contains 99 additional data points with the same structure as the original "surveys.csv".

In [ ]:
# load the csvs into DataFrames
survey_dataframe = pd.read_csv("data/surveys.csv")
survey2_dataframe = pd.read_csv("data/surveys2.csv")

# print out the dimensions of each DataFrame
print("""
Survey1: %s
Survey2: %s
""" % (survey_dataframe.shape, survey2_dataframe.shape))

### Concatenation

First, we'll look at concatenating two DataFrames. Often, data is separated over multiple files for many reasons including logical separation (time-based, etc), or due to size constraints, so it is useful to know how to concatenate multiple sets of data.

Pandas provides the "pd.concat()" method, which accepts a sequence (example: a tuple, list, dict) of Series or DataFrame objects to concatenate together. We'll pass a list containing the survey_dataframe and the survey2_dataframe to create a new DataFrame "big_survey_dataframe":

In [ ]:
big_survey_dataframe = pd.concat([survey_dataframe, survey2_dataframe])
big_survey_dataframe.shape 

In [ ]:
big_survey_dataframe.head(10)

### Merge
(http://pandas.pydata.org/pandas-docs/stable/merging.html)

Merging in Pandas is when you combine two different DataFrames that share a common identifier into a single DataFrame. This can be compared to the idea of joins in SQL. When you have two different DataFrames containing a shared identifier (think foreign key), you can use the "pd.merge()" function to merge them into a single DataFrame. There are several different ways you can merge the data, favoring different aspects of each dataset when deciding how to combine the data. We'll look at the two most popular and then provide a link with more information about additional types of merges.

We'll first load another dataset from "species.csv", containing information which expands on the species_id in our survey dataset.

In [ ]:
species_dataframe = pd.read_csv("data/species.csv")
species_dataframe.head(10)

#### Inner join merge

The first type of merge we'll look at is an inner join merge, and is the default type of merge in Pandas. As mentioned earlier, the two datasets to be merged need to have a common identifier in order to facilitate the merge. In the "surveys" and "species" datasets, this identifier is "species_id". The merge will connect the two datasets by this identifier, so we'll end up with one dataset containing both observation information, as well as augmented information about each specific species.

The idea of an inner join merge is that, for each row of data, the identifier that we are joining on (species_id) must exist in both datasets in order for the join to happen for that row. For example, if in our first row of data in "surveys", we have a species_id of "AX", but that species ID does not appear anywhere in the "species" dataset, that row will not be included in the final merged table.

The concept of an inner join is illustrated by the following Venn diagram:
<br><br><br>
<img src="http://bit.ly/2moPiCU"/>
<br><br><br>
To do an inner join merge in Pandas, use the "pd.merge()" method, and pass the following arguments:


| Argument Name | Description |
|----------|--------------------------------------------------------------------------------------------------------------------------------------------------|
| left     | the first DataFrame to merge, in our case will be the "survey" DataFrame      |                           
| right    | the second DataFrame to merge, in our case will be the "species" DataFrame                                                                       |
| left_on  | the identifier from the first DataFrame to use to merge -- ours will be "species_id"                                                                  |
| right_on | the identifier from the second DataFrame to use to merge -- ours will have the same name as the identifier in the first DataFrame, "species_id", but sometimes the column name could be different in each dataset |

<br>

We'll do a inner join merge, and look at the resulting DataFrame as well as the resulting size as compared to the size of the original DataFrames.

In [ ]:
merged_survey_dataframe = pd.merge(left=big_survey_dataframe, right=species_dataframe, left_on='species_id', right_on='species_id')
merged_survey_dataframe.head()

In [ ]:
# show the shape of each DataFrame involved in the merge and the resulting DataFrame
print("""
big_survey_dataframe:    %s
species_dataframe:       %s
merged_survey_dataframe: %s
""" % (big_survey_dataframe.shape, species_dataframe.shape, merged_survey_dataframe.shape))

#### Left join merge

Recall that in an inner join merge, the identifier that joins the datasets must appear in both datasets in order to be in the resulting merged dataset.

In a left join merge, this is not true, and the data in the first (or left) dataset will be favored. For each row in the first dataset, if the identifier doesn't appear in the second dataset, the row will still be included in the merged set, but the data will all be "NaN" (or null) for all columns that originally appeared in the second dataset. 

Specifically, in terms of our data, if the species_id was "AX" in the first dataset, and that didn't appear anywhere in the second dataset, the resulting merged row would contain "NaN" (or null) values for "genus", "species", and "taxa", since that information wasn't available. Contrast this to an inner join, where that row would never be included in the final merged DataFrame.

The concept of a left join is illustrated by this Venn diagram:
<br><br><br>
<img src="http://bit.ly/2lzkBvq"/>
<br><br><br>
Doing a left join merge in Pandas is almost the same as the inner join, except an additional argument called "how", specifying how to do the merge, must be passed into the "pd.merge()" method. In this case the value for that argument would be "left".

We'll do a left join merge, and look at the resulting DataFrame as well as the resulting size as compared to the size of the original DataFrames.

In [ ]:
left_merged_survey = pd.merge(left=big_survey_dataframe,
                              right=species_dataframe,
                              how='left',
                              left_on='species_id',
                              right_on='species_id')
left_merged_survey.head(10)

In [ ]:
# show the shape of each DataFrame involved in the merge and the resulting DataFrame
print("""
big_survey_dataframe:    %s
species_dataframe:       %s
left_merged_survey:      %s
""" % (big_survey_dataframe.shape, species_dataframe.shape, left_merged_survey.shape))

## Cleaning Data

We know that data comes to us in all shapes, sizes, formats, and with varying quality. Cleaning data is a common task in data analysis, and Pandas provides us with several nice methods that make doing that easier. We'll explore a couple useful/interesting examples in this section.

### Removing duplicate entries
(http://pandas.pydata.org/pandas-docs/version/0.17.1/generated/pandas.DataFrame.drop_duplicates.html)

Sometimes you'll run into the situation where you need to remove duplicate rows from a dataset. Pandas provides a convenience method for doing this on the DataFrame class called "drop_duplicates()". By default, this method will consider a row where all column values are the same to be a duplicate row. This may not always be the case though, and you may want to only consider a row a duplicate if only a subset of the column values are the same (or maybe only one). You can pass in an argument called "subset" in this case, and provide the column name(s) that you'd like to be considered in defining a duplicate. 

In our "left_merged_survey" DataFrame from the last section, there are duplicate values. We can verify this by calling the "duplicated()" method on "left_merged_survey", which returns a Series that will have a True or False value for each row, depending on if it is a duplicate or not. By using the "value_counts()" method on that result, we can see how many rows are duplicates:


In [ ]:
left_merged_survey.duplicated().value_counts()

We see that there are some rows that have a value of True, which means they are duplicates. We'll call the "drop_duplicates()" with no arguments to remove these, and we should be left with a DataFrame that has the same number of rows that were False (not duplicates) above:

In [ ]:
no_duplicates_survey = left_merged_survey.drop_duplicates()
no_duplicates_survey.shape

### Removing NaN (null) entries
(http://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.DataFrame.dropna.html)

In certain cases, you may want to remove rows from a dataset that contain null values. Pandas provides a method on DataFrames called "dropna()" whose default behavior is to drop the rows that have any NaN/null values. We'll call this on our "no_duplicates_survey" DataFrame. Notice how many rows were dropped - we had a lot of null values in this dataset:

In [ ]:
no_nan_survey = no_duplicates_survey.dropna()
no_nan_survey.shape

## Sorting/Grouping Data

### Sort
(http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sort_values.html)

Pandas provides a method called "sort_values()", which allows you to sort on either axis, and in ascending or descending order. By default, sort_values() will sort in ascending order. The only required argument is the value(s) you wish to sort by, which can be passed as either a string, or a list of strings. If you pass a list of strings, the DataFrame will be sorted by each value in the order they are specified in the list.

In our example, we sort our survey data in ascending order first by sex, then species, then weight, then hindfoot_length:

In [ ]:
sort_asc = no_nan_survey.sort_values(['sex','species','weight','hindfoot_length'])
sort_asc.head(10)

To sort in descending order, set the value of the "ascending" argument to False. This time we are sorting the same data by weight, then hindfoot_length:

In [ ]:
sort_desc = no_nan_survey.sort_values(['weight','hindfoot_length'], ascending = False)
sort_desc.head(10)

### Group By
(http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html)

Often when exploring or analyzing data, you'll want to group the data by a specific variable. In this case, one interesting way to group data would be by sex. Pandas provides a method on DataFrame called "groupby()", which will do that for you. In the following example, we group by sex, and get a count of all entries that are male vs female in the data. We'll see more interesting uses of "groupby()" later on.

In [ ]:
group_by_sex = sort_asc.groupby('sex')
group_by_sex.size()

## Transforming Data

Intro to Transforming Data section here

### Apply
(http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.apply.html)

Pandas allows us to run a function over each row to clean a string, or in the case of our example, apply a conversion formula to each value. We can use apply() to process specific rows or columns.

In [ ]:
# For example, you can process every value in a column through a function:
    
def convert_grams_to_oz(w):
    return w / 28.34952
    
no_nan_survey['weight'].apply(convert_grams_to_oz).head(10)

### DateTime

Pandas provides a DateTime datatype, which allows for more advanced filtering/sorting on dates, plus additional capabilities in terms of visualizing and working with time-series data.

If you have date information within your dataset, it is easy in Pandas to coerce it into DateTime format using the "pd.to_datetime()" function.

In [ ]:
# fails because dates are incorrect
"""datetimes = pd.to_datetime(
    { 'year': no_nan_survey.year,
      'month': no_nan_survey.month,
      'day': no_nan_survey.day
    })
    """

In [ ]:
# coerce to ignore incorrect dates
datetimes = pd.to_datetime(
    { 'year': no_nan_survey.year,
      'month': no_nan_survey.month,
      'day': no_nan_survey.day
    },
    errors='coerce')

dates_survey = no_nan_survey.assign(date = datetimes)
dates_survey.head(10)

In [ ]:
# see which dates were incorrect
dates_survey[(pd.isnull(dates_survey['date']))].head(10)

## Querying by DateTime

In [ ]:
dates_survey[(dates_survey['date'] > '1987-01-01') & (dates_survey['date'] < '1987-12-31')]

## Visualization

The final step in analyzing data is typically visualization, and this helps us communicate the meaning of the data or the trends found within the data to a broader audience. Pandas provides built-in methods to do visualization, but the Pandas API is still being standardized, so many types of visualizations are more easily created using other libraries. We will explore some examples using Pandas, and some examples using a different visualization library in Python called "ggplot". This exemplifies the strength of the compatibility of Pandas' data structures with other Python libraries.

#### Pandas Visualization Documentation
http://pandas.pydata.org/pandas-docs/version/0.18.1/visualization.html
ttp://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.plot.html

#### ggplot Visualization Documentation
http://ggplot.yhathq.com/docs/index.html

### Histograms
(https://en.wikipedia.org/wiki/Histogram)
(http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.plot.hist.html)

A histogram is a diagram that shows a history presentation of the distribution of numerical data. It is an estimate of the probability distribution of a continuous variable (quantitative variable).

We'll create a histogram in Pandas with our species/survey data to see the distribution of weight over the various species types. To create a histogram, use the ".plot.hist()" method. In our example, we pass an additional argument "orientation" that will control the horizontal/vertical orientation of the resulting graph.

In [ ]:
dates_survey.groupby('species')['weight'].plot.hist(orientation='horizontal')

### Bar Chart
(https://en.wikipedia.org/wiki/Bar_chart)

Still todo...

In [ ]:
#dates_survey.plot.bar(y='weight', x='species')

### Scatter Plot
(https://en.wikipedia.org/wiki/Scatter_plot)

A scatter plot is a type of plot that uses Cartesian coordinates to display values for typically two variables in a set of data. If the points are color-coded, one additional variable can be displayed. The data is displayed as a collection of points, each having the value of one variable determining the position on the horizontal axis and the value of the other variable determining the position on the vertical axis.

We'll create a scatter plot in Pandas to visualize the relationship between hindfoot_length and weight for the "flavus" species. We'll also color the points dark blue. 

In [ ]:
ax = dates_survey[(dates_survey['species'] == 'flavus')].plot.scatter(x='weight', y='hindfoot_length', color='DarkBlue', label='flavus')
ax

You can also create scatter plots with multiple sets of points that are color coded, as mentioned in the definition above. We'll create another scatter plot that contains our original points in blue, but also the same measures for the species "megalotis" in green:

In [ ]:
dates_survey[(dates_survey['species'] == 'megalotis')].plot.scatter(x='weight', y='hindfoot_length', color='DarkGreen', label='megalotis', ax=ax)

### Using ggplot for visualizations

Making plots is a very repetitive: draw this line, add these colored points, then add these, etc. Instead of re-using the same code over and over, ggplot implements them using a high-level but very expressive API. The result is less time spent creating your charts, and more time interpreting what they mean.

ggplot is not a good fit for people trying to make highly customized data visualizations. While you can make some very intricate, great looking plots, ggplot sacrifices highly customization in favor of general doing "what you'd expect".

#### Data

ggplot can use Pandas DataFrames as input for creating visualizations, and integrates seamlessly with Pandas.

#### Aesthetics (aes)

Aesthetics describe how your data will relate to your plots. Some common aesthetics are: x, y, and color. Aesthetics are specific to the type of plot (or layer) you're adding to your visual. For example, a scatterplot (geom_point) and a line (geom_line) will share x and y, but only a line chart has a linetype aesthetic.

#### Layers

ggplot lets you combine or add different types of visualization components (or layers) together. This is done by concatenating the different components together. You will typically start by defining the data and aesthetics (x and y axes, etc) in a "ggplot" object, and then concatenating additional layers, including the type of visualization, facets, etc. This is done using Python concatenation very similarly to how you'd concatenate multiple strings (+). 

### Box Plots
(from: http://flowingdata.com/2008/02/15/how-to-read-and-use-a-box-and-whisker-plot/)

The box-and-whisker plot is an exploratory graphic, created by John W. Tukey, used to show the distribution of a dataset (at a glance). Think of the type of data you might use a histogram with, and the box-and-whisker (or box plot, for short) could probably be useful.

#### Reading a Box-and-Whisker Plot
Let’s say we ask 2,852 people how many hamburgers they’ve consumed in the past week. We’ll sort those responses from least to greatest and then graph them with our box-and-whisker.

Take the top 50% of the group (1,426) who ate more hamburgers; they are represented by everything above the median (the white line). Those in the top 25% of hamburger eating (713) are shown by the top “whisker” and dots. Dots represent those who ate a lot more than normal or a lot less than normal (outliers). If more than one outlier ate the same number of hamburgers, dots are placed side by side.

#### Find Skews in the Data
The box-and-whisker shows you more than just four split groups. You can also see which way the data sways. For example, if there are more people who eat a lot of burgers than eat a few, the median is going to be higher or the top whisker could be longer than the bottom one. Basically, it gives you a good overview of the data’s distribution.

<img src="http://i1.wp.com/flowingdata.com/wp-content/uploads/2008/02/box-plot-explained.gif?w=1090"/>

We'll use ggplot to create a box plot of distribution of hindfoot length by species:

In [ ]:
ggplot(aes(x='species_id', y='hindfoot_length'), data=dates_survey) + geom_boxplot()

### Line Chart (faceted)
https://github.com/yhat/ggpy/blob/master/docs/how-to/Building%20Faceted%20(or%20Trellised)%20Plots.ipynb
http://ggplot.yhathq.com/docs/geom_density.html

Line charts (called density plots in ggplot) are easy to create. You can also create "small multiples" using ggplot's "facet_wrap()" method. Note that in the following example, we separate out the call to create the visualizaton over multiple lines, which is why we end each line with a "\".

In [ ]:
ggplot(dates_survey, aes(x='weight', fill='hindfoot_length')) +\
    geom_density(alpha=0.25) +\
    facet_wrap('species')

### Histogram with Faceting

We'll create another histogram, this time with ggplot and we'll also apply the faceting that we saw in the last example. We'll look at the distribution of weights, but this time for each different genus type. We'll also change the theme - ggplot comes with several different themes which change the look and feel of the resulting visualization. For more information about themes, check the "Themes" section on the ggplot documetation page (http://ggplot.yhathq.com/docs/index.html)


In [ ]:
ggplot(aes(x='weight'), data=dates_survey) + \
    geom_histogram(binwidth=20) + \
    facet_wrap("genus") + \
    theme_bw()